<a href="https://colab.research.google.com/github/arinafil/informatic_ds/blob/main/Johny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

# Прочтение таблиц

In [5]:
df = pd.read_excel('/content/Johny.xlsx')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,3243,0.003243,TGCGCCAGCAGGGTGGGACTAGCGGGAGGGCCTGTAGATGAGCAGT...,CASRVGLAGGPVDEQFF,TRBV5-1,TRBJ2-1
1,1,2305,0.002305,TGCGCCAGCAGCTTGGGAGGGGATACGCAGTATTTT,CASSLGGDTQYF,TRBV5-1,TRBJ2-3
2,2,1898,0.001898,TGTGCCAGCAGCTTAGTAAGCGGGAGGGTCACAGATACGCAGTATTTT,CASSLVSGRVTDTQYF,TRBV11-2,TRBJ2-3
3,3,1845,0.001845,TGTGCCAGCAGCTCCTATGAATCCCCCTACAATGAGCAGTTCTTC,CASSSYESPYNEQFF,TRBV11-2,TRBJ2-1
4,4,1296,0.001296,TGTGCCAGCAGCTTTTTGTCCAGTGAAGCTTTCTTT,CASSFLSSEAFF,TRBV7-2,TRBJ1-1


In [6]:
database = pd.read_excel('/content/vdjdb_slim.xlsx')

In [7]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [8]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,3243,0.003243,TGCGCCAGCAGGGTGGGACTAGCGGGAGGGCCTGTAGATGAGCAGT...,CASRVGLAGGPVDEQFF,TRBV5-1,TRBJ2-1
1,1,2305,0.002305,TGCGCCAGCAGCTTGGGAGGGGATACGCAGTATTTT,CASSLGGDTQYF,TRBV5-1,TRBJ2-3
2,2,1898,0.001898,TGTGCCAGCAGCTTAGTAAGCGGGAGGGTCACAGATACGCAGTATTTT,CASSLVSGRVTDTQYF,TRBV11-2,TRBJ2-3
3,3,1845,0.001845,TGTGCCAGCAGCTCCTATGAATCCCCCTACAATGAGCAGTTCTTC,CASSSYESPYNEQFF,TRBV11-2,TRBJ2-1
4,4,1296,0.001296,TGTGCCAGCAGCTTTTTGTCCAGTGAAGCTTTCTTT,CASSFLSSEAFF,TRBV7-2,TRBJ1-1
...,...,...,...,...,...,...,...
9995,8432,5,0.000005,TGTGCCAGCAGTCGACAGGGAGATAACACAGATACGCAGTATTTT,CASSRQGDNTDTQYF,TRBV12-3,TRBJ2-3
9996,7573,5,0.000005,TGTGCCAGCAGTCGACAGGGCCAACTTCCCCCCAACACTGAAGCTT...,CASSRQGQLPPNTEAFF,TRBV2,TRBJ1-1
9997,10499,5,0.000005,TGTGCCAGCAGTCGACAGGGGCAGCCCCTCCACTTT,CASSRQGQPLHF,TRBV12-4,TRBJ1-6
9998,9981,5,0.000005,TGTGCCAGCAGTCGACAGGGGGCATATGGCTACACCTTC,CASSRQGAYGYTF,TRBV12-4,TRBJ1-2


# Функция кластеризации

In [9]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [10]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,3243,0.003243,TGCGCCAGCAGGGTGGGACTAGCGGGAGGGCCTGTAGATGAGCAGT...,CASRVGLAGGPVDEQFF,TRBV5-1,TRBJ2-1,1
1,1,2305,0.002305,TGCGCCAGCAGCTTGGGAGGGGATACGCAGTATTTT,CASSLGGDTQYF,TRBV5-1,TRBJ2-3,2
2,2,1898,0.001898,TGTGCCAGCAGCTTAGTAAGCGGGAGGGTCACAGATACGCAGTATTTT,CASSLVSGRVTDTQYF,TRBV11-2,TRBJ2-3,3
3,3,1845,0.001845,TGTGCCAGCAGCTCCTATGAATCCCCCTACAATGAGCAGTTCTTC,CASSSYESPYNEQFF,TRBV11-2,TRBJ2-1,4
4,4,1296,0.001296,TGTGCCAGCAGCTTTTTGTCCAGTGAAGCTTTCTTT,CASSFLSSEAFF,TRBV7-2,TRBJ1-1,5
...,...,...,...,...,...,...,...,...
9995,8432,5,0.000005,TGTGCCAGCAGTCGACAGGGAGATAACACAGATACGCAGTATTTT,CASSRQGDNTDTQYF,TRBV12-3,TRBJ2-3,8696
9996,7573,5,0.000005,TGTGCCAGCAGTCGACAGGGCCAACTTCCCCCCAACACTGAAGCTT...,CASSRQGQLPPNTEAFF,TRBV2,TRBJ1-1,8697
9997,10499,5,0.000005,TGTGCCAGCAGTCGACAGGGGCAGCCCCTCCACTTT,CASSRQGQPLHF,TRBV12-4,TRBJ1-6,8698
9998,9981,5,0.000005,TGTGCCAGCAGTCGACAGGGGGCATATGGCTACACCTTC,CASSRQGAYGYTF,TRBV12-4,TRBJ1-2,1106


# Подсчёт Total Read Proportion

In [11]:
# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})


# Сортировка по Sequence Count

In [12]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
1,2,0.005045,220
86,89,0.001010,106
134,137,0.000462,51
195,199,0.000489,49
27,30,0.000662,40
...,...,...,...
3059,3068,0.000009,1
3057,3066,0.000009,1
3056,3065,0.000009,1
3055,3064,0.000009,1


# Сортировка по Total Read Proportion

In [13]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
1,2,0.005045,220
86,89,0.001010,106
6,7,0.000989,2
27,30,0.000662,40
195,199,0.000489,49
...,...,...,...
6861,6873,0.000010,2
8316,8328,0.000010,2
8128,8140,0.000010,2
5465,4919,0.000006,2


# Выбор кластера

In [14]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [15]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,2,CASSLGGDTQYF,0.002305,TRBV5-1,TRBJ2-3,TRBV28*01,TRBJ2-1*01,MHCI,HomoSapiens,HIV-1,Pol
1,22,CASSLSTDTQYF,0.000158,TRBV7-6,TRBJ2-3,TRBV12-3*01,TRBJ2-3*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
2,22,CASSLSTDTQYF,0.000158,TRBV7-6,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
3,22,CASSLSTDTQYF,0.000158,TRBV7-6,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
4,22,CASSLSTDTQYF,0.000158,TRBV7-6,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA3A
...,...,...,...,...,...,...,...,...,...,...,...
680,137,CASSPNTGELFF,0.000005,TRBV27,TRBJ2-2,"TRBV5-1*01,TRBV6-6*01,TRBV18*01",TRBJ2-2*01,MHCI,HomoSapiens,CMV,pp65
681,1015,CASSPGLAGDNEQFF,0.000005,TRBV6-4,TRBJ2-1,TRBV5-5*01,TRBJ2-1*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
682,1015,CASSPGLAGDNEQFF,0.000005,TRBV6-4,TRBJ2-1,TRBV6-3*01,TRBJ2-1*01,MHCI,HomoSapiens,CMV,IE1
683,1015,CASSPGLAGDNEQFF,0.000005,TRBV6-4,TRBJ2-1,TRBV6-3*01,TRBJ2-1*01,MHCI,HomoSapiens,EBV,EBNA4


In [16]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
7,156,CSVGTGGTNEKLFF,0.000083,TRBV29-1,TRBJ1-4,TRBV29-1*01,TRBJ1-4*01,MHCI,HomoSapiens,EBV,BMLF1
8,156,CSVGTGGTNEKLFF,0.000083,TRBV29-1,TRBJ1-4,TRBV29-1*01,TRBJ1-4*01,MHCII,HomoSapiens,InfluenzaA,NP
9,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
10,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
11,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
...,...,...,...,...,...,...,...,...,...,...,...
646,30,CASSLGGNEQFF,0.000005,TRBV7-9,TRBJ2-1,TRBV7-9*01,TRBJ2-1*01,MHCI,HomoSapiens,EBV,BZLF1
649,30,CASSLGGNEQFF,0.000005,TRBV7-9,TRBJ2-1,"TRBV7-9*03,TRBV11-1*01",TRBJ2-1*01,MHCI,HomoSapiens,HCV,NS5B
653,89,CASSLSGNTEAFF,0.000005,TRBV7-9,TRBJ1-1,TRBV7-9*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
672,199,CASSHSYEQYF,0.000005,TRBV12-4,TRBJ2-7,TRBV12-3*01,TRBJ2-7*01,MHCI,HomoSapiens,HomoSapiens,MLANA


In [21]:
full_vj.value_counts()

Cluster  CDR3.amino.acid.sequence  Read.proportion  bestVGene  bestJGene  v.segm      j.segm      mhc.class  species      antigen.species  antigen.gene
160      CASSLEGDQPQHF             0.000055         TRBV5-1    TRBJ1-5    TRBV5-1*01  TRBJ1-5*01  MHCI       HomoSapiens  CMV              IE1             2
                                                                                                                          InfluenzaA       NP              2
                                                                                                                          SARS-CoV-2       Spike           2
                                   0.000082         TRBV5-1    TRBJ1-5    TRBV5-1*01  TRBJ1-5*01  MHCI       HomoSapiens  CMV              IE1             1
                                                                                                                          InfluenzaA       NP              1
                                                                                                                          SARS-CoV-2       Spike           1
Name: count, dtype: int64

# Обработка для одного выбранного кластера

In [17]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 160]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
161,162,82,0.000082,TGCGCCAGCAGCTTGGAGGGTGATCAGCCCCAGCATTTT,CASSLEGDQPQHF,TRBV5-1,TRBJ1-5,160
276,282,55,0.000055,TGCGCCAGCAGCTTGGAAGGGGACCAGCCCCAGCATTTT,CASSLEGDQPQHF,TRBV5-1,TRBJ1-5,160
277,281,55,0.000055,TGCGCCAGCAGCTTGGAGGGAGATCAGCCCCAGCATTTT,CASSLEGDQPQHF,TRBV5-1,TRBJ1-5,160
7070,6879,6,0.000006,TGTGCCAGCAGTTTAAACGGAGATCAGCCCCAGCATTTT,CASSLNGDQPQHF,TRBV12-3,TRBJ1-5,160


In [18]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [19]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
1,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
2,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
3,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
4,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
5,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
6,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
7,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
8,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike


# Разборки с V и J

In [20]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
1,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
2,160,CASSLEGDQPQHF,0.000082,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
3,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
4,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
5,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
6,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
7,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
8,160,CASSLEGDQPQHF,0.000055,TRBV5-1,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,SARS-CoV-2,Spike
